<a href="https://colab.research.google.com/github/AnnJer/Practice_in_Data_Preprocessing/blob/master/stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#загрузка данных
from google.colab import files

train = files.upload()
test= files.upload()

# train = pd.read_csv('/home/ann/task/train.tsv', sep='\t')
# test = pd.read_csv('/home/ann/task/dev.tsv', sep='\t')
train = train[:100000]
train_X = train.drop(['click', 'id'], axis=1)
train_y = train['click']

test_X = train.drop(['click', 'id'], axis=1)
test_y = train['click']

Saving dev.tsv to dev.tsv


TypeError: ignored

In [0]:
import pandas as pd
import numpy as np

%pylab inline

plt.style.use('default')
figsize(12, 9)

import warnings
warnings.filterwarnings("ignore")


pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rcParams['font.family'] = 'Ubuntu'

plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('font', weight='bold')
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)

from matplotlib import rc
font = {'family': 'Vernada',
        'weight': 'normal',
        'size': 10}
rc('font', **font)

Populating the interactive namespace from numpy and matplotlib


In [0]:
train['click'].mean()

In [0]:
test['click'].mean()

In [0]:
#объединить все в один датасет
data = pd.concat([train, test])
data = data.iloc[:10000]

data.describe()
data.shape

In [0]:
data['click'].mean()

In [0]:
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split

class Stacking(BaseEstimator, ClassifierMixin):
    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

In [0]:
from sklearn.metrics import roc_auc_score

def run_and_plot(clf, X, y, label):
    # plt.figure(figsize=(6, 5))
    
    a = clf.predict(X)
    print (label + ' AUC-ROC  = ' + str( roc_auc_score(y, a) ))
#     xx, yy = np.meshgrid(np.linspace(-3, 3, 500), np.linspace(-3, 3, 500))

    
    

#     Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
#     Z = Z.reshape(xx.shape)

#     plt.figure(figsize=(8, 7))
#     plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), Z.max(), 20), cmap=plt.cm.bwr, alpha=0.3) # plt.cm.Blues_r cmap=plt.cm.Blues_r)
#     #a_ = plt.contour(xx, yy, Z, levels=[threshold], linewidths=1, colors='black')
#     #plt.contourf(xx, yy, Z, levels=[threshold, Z.max()], colors='#CCDDFF')
    
#     # не всё... :100
#     plt.scatter(X[:300, 0], X[:300, 1], c=y[:300], s=20, alpha=1.0)
#     plt.xlim([-3, 3])
#     plt.ylim([-3, 3])
#     # plt.axis('tight')
#     plt.axis('off')
#     plt.title(label)

In [0]:
X = data.drop(['click', 'id'], axis=1)
y = data['click']

# train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, random_state=1999)

In [0]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(max_depth=5, random_state=17)

tree_params = {'max_depth': range(1,11),
               'max_features': range(4,19)}

tree_grid = GridSearchCV(tree, tree_params,
                         cv=5, n_jobs=-1,
                        verbose=True)

tree_grid.fit(train_X, train_y)
tree_grid.best_params_

In [0]:
tree_grid.best_score_

In [0]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor


knn1 = KNeighborsRegressor(n_neighbors=3)
knn1.fit(train_X, train_y)
run_and_plot(knn1, test_X, test_y, '3NN')

# knn2 = KNeighborsRegressor(n_neighbors=10)
# knn2.fit(train_X, train_y)
# run_and_plot(knn2, test_X, test_y, '10NN')


# rg0 = Ridge(alpha=0.01)
# rg0.fit(train_X, train_y)
# run_and_plot(rg0, test_X, test_y, 'ridge-0.01')

# rg1 = Ridge(alpha=1.1)
# rg1.fit(train_X, train_y)
# run_and_plot(rg1, test_X, test_y, 'ridge-1.1')

# rg2 = Ridge(alpha=100.1)
# rg2.fit(train_X, train_y)
# run_and_plot(rg2, test_X, test_y, 'ridge-100.1')


# rf1 = RandomForestRegressor(n_estimators=100, max_depth=1)
# rf1.fit(train_X, train_y)
# run_and_plot(rf1, test_X, test_y, 'rf-d1')

# rf2 = RandomForestRegressor(n_estimators=100, max_depth=5)
# rf2.fit(train_X, train_y)
# run_and_plot(rf2, test_X, test_y, 'rf-d5')

# rf3 = RandomForestRegressor(n_estimators=100, max_depth=8)
# rf3.fit(train_X, train_y)
# run_and_plot(rf3, test_X, test_y, 'rf-d8')


# gbm1 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=2, n_estimators=200, nthread=-1, objective='regression')    
# gbm1.fit(train_X, train_y)
# run_and_plot(gbm1, test_X, test_y, 'gbm-d2')

# gbm2 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=5, n_estimators=200, nthread=-1, objective='regression')    
# gbm2.fit(train_X, train_y)
# run_and_plot(gbm2, test_X, test_y, 'gbm-d5')

In [0]:
3NN AUC-ROC  = 0.6499584884107964
10NN AUC-ROC  = 0.7143468402021302
ridge-0.01 AUC-ROC  = 0.7306063339687726
ridge-1.1 AUC-ROC  = 0.7281704830852095
ridge-100.1 AUC-ROC  = 0.7138504032014039
rf-d1 AUC-ROC  = 0.5679338575708062
rf-d5 AUC-ROC  = 0.7601675025720164
rf-d8 AUC-ROC  = 0.7570234015674171
gbm-d2 AUC-ROC  = 0.7631314678649238
gbm-d5 AUC-ROC  = 0.7579671046659404


In [0]:
models = [knn1, knn2,rg1, rg2, rf1, rf2,rf3, gbm1, gbm2] # , 
ens_model = Ridge()
s1 = Stacking(models, ens_model)
s1.fit(train_X, train_y)
run_and_plot(s1, test_X, test_y, '1-stacking')


s2 = Stacking(models, ens_model)
s2.fit(train_X, train_y, p=0.3)
run_and_plot(s1, test_X, test_y, '2-stacking')



In [0]:
from sklearn.model_selection import KFold # cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from time import time

tm = time()

cv = KFold(n_splits=5, shuffle=True, random_state=1) # как тестировать

rf = RandomForestClassifier(n_estimators=10, max_features=5, n_jobs=-1, random_state=1) # алгоритм

tmp = cross_val_score(rf, X, y, scoring='roc_auc', cv=cv) # AUC ROC!!!

print (tmp)
print (u' = %s,  = %s,  = %s' % (mean(tmp), std(tmp), time() - tm))

plt.figure(figsize=(12,4))
plt.bar(np.arange(len(tmp)), tmp, color='#000099')
plt.ylim([min(tmp)-0.001, max(tmp) + 0.001])


In [0]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from time import time

tm = time()

cv = ShuffleSplit(n_splits=5, test_size=0.75, train_size=None, random_state=1)


# rf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, min_samples_split=2,
#                        min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=5,
#                        max_leaf_nodes=None, min_impurity_decrease=1e-07, bootstrap=True, oob_score=False,
#                        n_jobs=-1, random_state=None, verbose=0, warm_start=False, class_weight=None)


gbm = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100,
                                 subsample=1.0, criterion='friedman_mse', min_samples_split=2,
                                 min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=4,
                                 min_impurity_decrease=1e-07, init=None, random_state=None,
                                 max_features=None, verbose=0, max_leaf_nodes=None,
                                 warm_start=False, presort='auto')

param_name = "n_estimators"


#pars = [1, 5, 10, 15, 20, 25, 30] # d=3 0.2 - 0.872
# pars = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
#pars = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
pars = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

train_errors, test_errors = validation_curve(gbm, X, y, param_name=param_name,
                                             param_range=pars, cv=cv.split(X), scoring='roc_auc', n_jobs=-1)

print ('time = ' + str(time() - tm))

figsize(12, 5)

plt.plot(pars, test_errors.mean(axis=1), label="cv", lw=2)

plt.fill_between(pars, test_errors.min(axis=1),
                 test_errors.max(axis=1), alpha=0.2,
                 color="darkorange", lw=2)

plt.fill_between(pars, test_errors.mean(axis=1) - test_errors.std(axis=1),
                 test_errors.mean(axis=1) + test_errors.std(axis=1), alpha=0.1,
                 color="darkblue", lw=1)

plt.legend(loc="best")
# plt.ylim([0.82, 0.88]) # 1.01
plt.ylabel('ROC AUC')
plt.xlabel(param_name)
